#Environment Setup

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Put Data in DataFrame (Articles)

In [4]:
import pandas as pd
import glob, os

# Place dataset path here
FNNpath = "../Datasets/FNN Titles/"
CoAIDpath = "../Datasets/CoAID/"


politiFactFake = pd.read_csv(FNNpath+"politifact_fake.csv", usecols=['title'])
politiFactFake['label']=0
politiFactTrue = pd.read_csv(FNNpath+"politifact_real.csv", usecols=['title'], nrows=len(politiFactFake.values))
politiFactTrue['label']=1

CoAIDFalse = pd.read_csv(CoAIDpath+"NewsFakeCOVID-19.csv", usecols=['title'])
CoAIDFalse['label']=0
CoAIDTrue = pd.read_csv(CoAIDpath+"NewsRealCOVID-19.csv", usecols=['title'], nrows=len(CoAIDFalse.values))
CoAIDTrue['label']=1


dfTotal = pd.concat([politiFactTrue, politiFactFake, CoAIDTrue, CoAIDFalse])


X = dfTotal['title'].values
y = dfTotal['label'].values
print("len(X)", len(X))
print("len(y)", len(y))

len(X) 2008
len(y) 2008


#Next Steps With Data


# Finetune BERT

In [5]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 7.9 MB/s 
     |████████████████████████████████| 140 kB 50.6 MB/s 
     |████████████████████████████████| 86 kB 3.8 MB/s 
     |████████████████████████████████| 212 kB 50.2 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 1.1 MB 39.0 MB/s 
     |████████████████████████████████| 596 kB 49.2 MB/s 
     |████████████████████████████████| 127 kB 51.2 MB/s 
     |████████████████████████████████| 144 kB 53.2 MB/s 
     |████████████████████████████████| 271 kB 51.1 MB/s 
     |████████████████████████████████| 94 kB 441 kB/s 
     |████████████████████████████████| 112 kB 55.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing ins

In [6]:
from datasets import Dataset
from sklearn.utils import shuffle
dfTotal = shuffle(dfTotal)
train_dataset = Dataset.from_pandas(dfTotal)

In [7]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

def tokenize_data(example):
    return tokenizer(example['title'], padding='max_length')

train_dataset = train_dataset.map(tokenize_data, batched=True)


from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", num_train_epochs=3)


from transformers import Trainer

trainer_train = Trainer(
    model=model, args=training_args, train_dataset=train_dataset
)

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [8]:
trainer_train.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, title. If __index_level_0__, title are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2008
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 753


Step,Training Loss
500,0.308000


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=753, training_loss=0.2521172633684014, metrics={'train_runtime': 353.1221, 'train_samples_per_second': 17.059, 'train_steps_per_second': 2.132, 'total_flos': 1584980997488640.0, 'train_loss': 0.2521172633684014, 'epoch': 3.0})

In [9]:
import torch
model_path = '../models/roberta-fake-news_on_CoAID&&PolitiFact_only.pt'
torch.save(model.state_dict(),model_path)

In [10]:
path_external = "/content/drive/MyDrive/CoVerifi&MedVerifi/" # Replace with appropriate path
dfTotal_external = pd.read_csv(path_external+"Combined_News.csv", usecols=['title', 'label'])

def binaryLabel(label):
  if label == "TRUE":
    return 1
  return 0
dfTotal_external = dfTotal_external.dropna()
dfTotal_external['label'] = dfTotal_external['label'].apply(lambda label: binaryLabel(str(label)))
dataset_external = Dataset.from_pandas(dfTotal_external)

In [12]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

dataset_external = dataset_external.map(tokenize_data, batched=True)

trainer_eval = Trainer(
    model=model,
    args=training_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
trainer_eval.evaluate()

  0%|          | 0/8 [00:00<?, ?ba/s]

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, title. If __index_level_0__, title are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7057
  Batch size = 8


{'eval_accuracy': 0.910726937792263,
 'eval_loss': 0.43977585434913635,
 'eval_runtime': 132.2031,
 'eval_samples_per_second': 53.38,
 'eval_steps_per_second': 6.679}